<h2> H₂O Prediction in Icelandic Melts Using Random Forest Regression </h2>

<h4>This notebook trains Random Forest models on seven oxide compositions (SiO₂, Al₂O₃, FeO, MgO, CaO, Na₂O, K₂O) of plagioclase-hosted, olivine-hosted and clinopyroxene-hosted melt inclusions to estimate H₂O content in volcanic glass samples and melt inclusions.</h4>

<h5> 
    
**Author:** Rahul Subbaraman

**Affiliation:** University of Manchester 

**Email:** rahul.subbaraman@manchester.ac.uk 

**Date:** 05-08-2025 
</h5>

<h3> IMPORTS & SETUP </h3>

<h5>Update the file paths - <b>Sample_Train_data.csv</b> and <b>Sample_Test_data.csv</b> with the user file names.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# ### Constants
Oxides = ['SiO2', 'Al2O3', 'FeO', 'MgO', 'CaO', 'Na2O', 'K2O']
train_data_path = os.path.join('Imports', "Sample_Train_data.csv")
test_data_path = os.path.join('Imports', "Sample_Test_data.csv")
Output_CSV = os.path.join('Exports', 'Multivar_Regr_H2O_with_RF.csv')
Model_file = os.path.join('Exports', 'best_rf_model.joblib')

missing_files = []
if not os.path.isfile(train_data_path):
    missing_files.append("Sample_Train_data.csv")
if not os.path.isfile(test_data_path):
    missing_files.append("Sample_Test_data.csv")

if not missing_files:
    print("Imports and setup done.")
else:
    print("Missing file(s):", ", ".join(missing_files))

Imports and setup done.


<h3> LOADING DATASETS </h3>

<h5> Ensure the format of the training file matches <b>Sample_Train_Data.csv</b> and test file matches <b>Sample_Test_Data.csv</b></h5>

<h4> LOAD TRAINING DATA </h4>

<h5>This model is not supposed to be calibrated with Matrix Glass. If the data has Matrix Glass, uncomment the 2nd line below.</h5>

In [2]:
df = pd.read_csv(train_data_path, low_memory=False)
# Uncomment the line below, if appropriate
#df = df[df['TYPE'] != 'Matrix Glass']  # exclude matrix glass from training.
print(f"Loaded training dataset with {len(df)} data points after filtering.")

Loaded training dataset with 7 data points after filtering.


<h4> LOAD TESTING DATA </h4>

In [4]:
gf = pd.read_csv(test_data_path, low_memory=False)
print(f"Loaded testing dataset with {len(gf)} data points.")

Loaded testing dataset with 4 data points.


<h3> RANDOM FOREST REGRESSION </h3>

<h4> DEFINE REGRESSION FUNCTION </h4>

In [5]:
def run_regression(group, location, typ, model_label, results):
    group_clean = group.dropna(subset=Oxides + ['H2O'])
    if len(group_clean) < 50:
        return

    X = group_clean[Oxides]
    y = group_clean['H2O']

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)

    results.append({
        'Location': location,
        'Type': typ,
        'Model': model_label,
        'N': len(y),
        'R2': r2,
        **dict(zip(Oxides, model.feature_importances_))
    })
print('Regression function defined')

Regression function defined


<h4> PERFORM REGRESSION BY LOCATION AND TYPE GROUPINGS </h4>

<h5>The regression is performed across different groupings of sample types and locations, identifying the best model based on <strong>R² values</strong> and the <strong>number of samples</strong>.</h5>

In [8]:
results = []

# By LOCATION + TYPE
temp = df.groupby(['LOCATION', 'TYPE'])
for (loc, typ), group in temp:
    run_regression(group, loc, typ, 'Multivariate (Loc+Type)', results)

# By TYPE only
temp = df.groupby('TYPE')
for typ, group in temp:
    run_regression(group, 'All', typ, 'Multivariate (By TYPE)', results)

# By LOCATION only
temp = df.groupby('LOCATION')
for loc, group in temp:
    run_regression(group, loc, 'All', 'Multivariate (By LOCATION)', results)
print(results[:3])

# Compile results
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by=['R2', 'N'], ascending=[False, False])
results_df.head()

print('Regression performed on the reference data')

[]


KeyError: 'R2'

<h4> RETRAIN BEST MODEL </h4>

In [ ]:
best_row = results_df.iloc[0]
print(f"Best model: Location = {best_row['Location']}, Type = {best_row['Type']}, R2 = {best_row['R2']:.3f}")

if best_row['Location'] == 'All':
    loc_filter = df['LOCATION'].notnull()
else:
    loc_filter = df['LOCATION'] == best_row['Location']

if best_row['Type'] == 'All':
    typ_filter = df['TYPE'].notnull()
else:
    typ_filter = df['TYPE'] == best_row['Type']

best_group = df[loc_filter & typ_filter].dropna(subset=Oxides + ['H2O'])
X_train = best_group[Oxides]
y_train = best_group['H2O']

best_model = RandomForestRegressor(n_estimators=100, random_state=42)
best_model.fit(X_train, y_train)
print("Best model retrained.")

<h5> Run the cells below as applicable </h5>

<h5> (a) Predict H2O on Matrix Glass Data </h5>

In [ ]:
matrix = gf[gf['PHASE'] == 'Matrix Glass'].copy()
matrix['H2O'] = best_model.predict(matrix[Oxides])
print('H2O Content of Matrix Glass is calculated')

<h5> (b) Predict H2O on Plagioclase-hosted Melt Inclusions (PHMI) Data </h5>

In [ ]:
PHMI = gf[gf['PHASE'] == 'PHMI'].copy()
PHMI['H2O'] = best_model.predict(PHMI[Oxides])
print('H2O Content of PHMI is calculated')

<h5> (c) Predict H2O on Olivine-hosted Melt Inclusions (OHMI) Data </h5>

In [ ]:
OHMI = gf[gf['PHASE'] == 'OHMI'].copy()
OHMI['H2O'] = best_model.predict(OHMI[Oxides])
print('H2O Content of OHMI is calculated')

<h5> (d) Predict H2O on Clinopyroxene-hosted Melt Inclusions (CHMI) Data </h5>

In [ ]:
CHMI = gf[gf['PHASE'] == 'CHMI'].copy()
CHMI['H2O'] = best_model.predict(CHMI[Oxides])
print('H2O Content of CHMI is calculated')

<h4> EXPORT RESULTS </h4>

In [ ]:
os.makedirs('Exports', exist_ok=True)
results_df.to_csv(Output_CSV, index=False)
print(f"Results saved to: {Output_CSV}")
dump(best_model, Model_file)
print(f"Best model saved to: {Model_file}")